# Déclarations

In [ ]:
import os
import sys
import nltk
import pandas as pd
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from gensim.models import word2vec
from nltk.corpus import stopwords 
from tensorflow.contrib.tensorboard.plugins import projector

tqdm.pandas()

# Chargement du fichier CSV

In [ ]:
trainFile = "../input/wiki_movie_plots_deduped.csv"

pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
df = pd.read_csv(os.path.basename(trainFile))
os.chdir(pwd)

In [ ]:
print("Nombre de lignes : {0}".format(len(df)))

# Visualisation du jeu de données

In [ ]:
dfAnalyze = df.copy()
dfAnalyze.head()

### Nombre de films par date 
Le jeu de données est ordonné par année de sortie (Release Year). De ce fait les 2000 premiers enregistrements pris pour les tests vont de 1900 à 1935 (car classés par date).  
Cela va avoir une incidence sur la cohérence des mots proches entre le jeu de données réduit et le jeu de données complet (détails plus bas).

In [ ]:
hist = dfAnalyze.plot.hist()

### Nombre de films par pays

In [ ]:
columns = ['Release Year', 'Director','Cast', 'Genre', 'Wiki Page', 'Plot']
dfPie = dfAnalyze.drop(columns, axis=1)

dfPie = dfPie.groupby(['Origin/Ethnicity']).count().rename(columns= {'Title':'count'})
pie = dfPie.plot.pie(subplots=True, figsize=(7, 7))

# Réduction du jeu de données pour les tests
Pour la visualisation du kernel sur Kaggle, j'ai réduit le jeu de données. Cela a une incidence sur les résultats. J'ai donc mis en commentaires plus bas les résultats propres à l'ensemble du jeu de données.

In [ ]:
df = df.head(2000)
print("Nombre de lignes sélectionnées : {0}".format(len(df)))

# Nettoyage
- Suppression des stop words anglais
- Suppression des chaînes de caractères de longueur inférieure ou égale à 2

In [ ]:
def tokenize_without_stop_words(text):
    sentence = nltk.word_tokenize(text.lower())
    sentence = [w for w in sentence if not w in stopwords.words('english')]
    sentence = [w for w in sentence if len(w) > 2 ]
    return sentence

sentences = df.progress_apply(lambda row: tokenize_without_stop_words(row['Plot']), axis=1)
sentences = sentences.tolist()

# Vectorization Word2Vc

## Quelques informations sur la vectorization

### Modèle
En utilisant tout le jeu de données, je ne garde que les mots présents au moins 750 fois (soit présent à 2%).  
Cela représente 1686 mots.  
Pour la visualisation le jeu de données est réduit et j'ai donc adapté le min_count.

In [ ]:
model = word2vec.Word2Vec(sentences, min_count=50, workers=4)
print(model)

# model = word2vec.Word2Vec(sentences, min_count=750, workers=4)
# Result Full Dataset : Word2Vec(vocab=1686, size=100, alpha=0.025)

### Visualisation du vocabulaire

In [ ]:
#for entry in sorted(model.wv.vocab):
#    print(entry)

### Tests de la cohérence en recherchant des mots proches
Attention car ces tests de cohérence différent en fonction du nombre de lignes prises du jeu de données.   
J'ai mis en commentaire les 5 mots les plus proches en utilisant tout le jeu de données.

In [ ]:
print(model.wv.most_similar(['suicide'], topn=5))
# Result Full Dataset : [('murder', 0.54288649559021), ('failed', 0.47224748134613037), ('rape', 0.4682120680809021), 
# ('murders', 0.4570953845977783), ('death', 0.3709148168563843)]

#### Mots proches de war
Attention car en ne prenant par exemple que les 2000 premiers enregistrements les résultats sont complètement différents : le mot war se rattache à la guerre entre la France et l'Allemagne.    
En revanche avec tout le jeu de données, le mot war se rattache aux Etats-Unis, Japon...  
Cela vient du fait que les enregistrements sont classés par date et que les films traitent souvent de l'actualité.

In [ ]:
print(model.wv.most_similar(['war'], topn=5))
# Result Full Dataset : [('u.s.', 0.597042441368103), ('army', 0.5714285969734192), ('china', 0.55967777967453), 
# ('union', 0.558294951915741), ('japan', 0.5436923503875732)]

# Transformation du modèle word2vec en modèle exploitable par tensorboard

Merci à BrikerMan (https://gist.github.com/BrikerMan/7bd4e4bd0a00ac9076986148afc06507)  
Avec l'apport personnel d'une correction (= purge des anciens nodes/graphes)

In [ ]:
def transform_word2vec_to_tensor(model, output_path):
    meta_file = "w2x_metadata.tsv"
    placeholder = np.zeros((len(model.wv.index2word), 100))

    with open(os.path.join(output_path,meta_file), 'wb') as file_metadata:
        for i, word in enumerate(model.wv.index2word):
            placeholder[i] = model[word]
            # temporary solution for https://github.com/tensorflow/tensorflow/issues/9094
            if word == '':
                print("Emply Line, should replaced by any thing else, or will cause a bug of tensorboard")
                file_metadata.write("{0}".format('<Empty Line>').encode('utf-8') + b'\n')
            else:
                file_metadata.write("{0}".format(word).encode('utf-8') + b'\n')

    # Correction : purge les anciens nodes/graphes
    tf.reset_default_graph()
    
    sess = tf.InteractiveSession()

    embedding = tf.Variable(placeholder, trainable = False, name = 'w2x_metadata')
    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(output_path, sess.graph)

    # adding into projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = 'w2x_metadata'
    embed.metadata_path = meta_file

    # Specify the width and height of a single thumbnail.
    projector.visualize_embeddings(writer, config)
    saver.save(sess, os.path.join(output_path,'w2x_metadata.ckpt'))

directory = './output/'
if not os.path.exists(directory):
    os.makedirs(directory)

# Pour la visualisation Kaggle, j'ai mis en commentaire transform_word2vec_to_tensor(model, directory). Il faut donc dé-commenter la ligne ci-dessous.
#transform_word2vec_to_tensor(model, directory)

# Lecture dans tensorboard
- Pour ma part, j'utilise anaconda. J'ai donc au préalable installé tensorboard (https://anaconda.org/conda-forge/tensorboard) avec tous ses dépendances.
- Ensuite je me suis placé dans le répertoire contenant mon notebook et j'ai tapé la commande :  
`tensorboard --logdir output --host localhost`  
Exemple : `tensorboard --logdir . --host localhost`
- J'ouvre mon navigateur et je charge la page dont l'url est indiquée suite à l'exécution de la commande dans la console anaconda.

### Vue 3D du mot "war" à partir des 2000 premiers enregistrements du jeu de données

![Small Dataset War](https://farm5.staticflickr.com/4805/44359366950_c42826e5e1_b.jpg)

### Vue 3D du mot "war" à partir de tout le jeu de données

![Full Dataset War](https://farm5.staticflickr.com/4850/45263897465_c06d96d2dc_b.jpg)

### Vue 3D du mot "suicide" à partir de tout le jeu de données

![Full Dataset Suicide](https://farm5.staticflickr.com/4841/32304879878_7836210f2a_z.jpg)